# TODO:
## 1. Сделать отдельный файл с токеном и добавить его в в gitignore, чтобы когда на гитхаб выкладывали не было видно какой у нас токен. Соответственно, проделать считывание токена из файла.
## 2. Сделать отдельный файл с весами нейронки и добавить его в в gitignore, чтобы когда на гитхаб выкладывали не было видно весов нейросети. Соответственно, проделать считывание весов из файла
 

In [1]:
import telebot
import logging
import configparser
from telebot import types


In [2]:
token = "5639499556:AAGvwXXkXJa8Uf0IAIQXZJrfjjNoWyC6bcQ"
bot = telebot.TeleBot(token);



logger = telebot.logger
telebot.logger.setLevel(logging.DEBUG) # Outputs debug messages to console.


launch_counter = 0

In [3]:
launch_counter +=1

@bot.message_handler(commands=['start','help'])
def send_usual_commands(message):
    if message.text == '/start':
        bot.reply_to(message, "Ok, lets start. To proccess photo enter: </pr>. For extra info see </help>")
    else:
        bot.reply_to(message, "This bot process images. To proccess photo enter: </pr>. For extra info see </help>")
        
# @bot.message_handler(commands=['help'])
# def send_help(message):
#     bot.reply_to(message, "This bot .... info about bot and its' commands")
    
@bot.message_handler(commands=['set_up'])
def send_settings(message):
    INSTRUCTION = "To update parameters press button on keyboard below."
    
    config = configparser.ConfigParser()
    config.read('config.ini')
    default_settings = dict(config['DEFAULT'])
    
    buttons_text = list(default_settings.keys())
    buttons_text.append("CANCEL")
    
    settings_str = ""
    for (key,item) in dict(default_settings).items():
        settings_str+=f"{key}: \t{item}\n"
    
    keyboard = types.InlineKeyboardMarkup()
    
    for btn_txt in buttons_text:
        callback_button = types.InlineKeyboardButton(text=btn_txt, callback_data=btn_txt)
        keyboard.add(callback_button)

    
    bot.reply_to(message, f"Bot current settings:\n{settings_str}\n{INSTRUCTION}",reply_markup=keyboard)
    #bot.send_message(message.chat.id, "Я – сообщение из обычного режима", reply_markup=keyboard)
   




@bot.callback_query_handler(func=lambda call: True)
def callback_inline(call):
    if call.message:
        if call.data == "visibility":
            bot.send_message(call.message.chat.id, f"Changing visibility. Enter num to replace visibility param:")
            bot.register_next_step_handler(call.message, get_visibility);
            print('visibility is chosen')
#             keyboard = types.InlineKeyboardMarkup(row_width=3)
#             a = types.InlineKeyboardButton(text=" ", callback_data="2")
#             b = types.InlineKeyboardButton(text=" ", callback_data="2")
#             c = types.InlineKeyboardButton(text=" ", callback_data="2")
#             d = types.InlineKeyboardButton(text=" ", callback_data="2")
#             e = types.InlineKeyboardButton(text=" ", callback_data="2")
#             f = types.InlineKeyboardButton(text=" ", callback_data="2")
#             g = types.InlineKeyboardButton(text=" ", callback_data="2")
#             h = types.InlineKeyboardButton(text=" ", callback_data="2")
#             i = types.InlineKeyboardButton(text=" ", callback_data="2")
#             keyboard.add(a, b, c, d, e, f, g, h, i)
#             bot.edit_message_text(chat_id=call.message.chat.id, message_id=call.message.message_id, text="X", reply_markup=keyboard)
        if call.data == "CANCEL":
            bot.send_message(call.message.chat.id, f"Action cancelled")
                
        if call.data == "3":
            keyboard = types.InlineKeyboardMarkup(row_width=3)
            a = types.InlineKeyboardButton(text=" ", callback_data="1")
            b = types.InlineKeyboardButton(text=" ", callback_data="1")
            c = types.InlineKeyboardButton(text=" ", callback_data="1")
            d = types.InlineKeyboardButton(text=" ", callback_data="1")
            e = types.InlineKeyboardButton(text="0", callback_data="1")
            f = types.InlineKeyboardButton(text=" ", callback_data="1")
            g = types.InlineKeyboardButton(text=" ", callback_data="1")
            h = types.InlineKeyboardButton(text=" ", callback_data="1")
            i = types.InlineKeyboardButton(text=" ", callback_data="1")
            keyboard.add(a, b, c, d, e, f, g, h, i)
            bot.edit_message_text(chat_id=call.message.chat.id, message_id=call.message.message_id, text="X", reply_markup=keyboard)
            
            

# bot.register_next_step_handler(message, update_settings);
@bot.message_handler(commands=['pr'])
def send_processing(message):
    bot.reply_to(message, "Send image to process..")
    bot.register_next_step_handler(message, get_image);
@bot.message_handler(content_types=['text'])
def get_unknown_message(message):
    bot.reply_to(message, "Unknown message. See </help> command")


#def update_settings(message):
def get_visibility(message):
    if message.content_type == 'text':
        try:
            new_value = float(message.text)
        except ValueError:
            bot.send_message(message.from_user.id, f"Thats not a num type. Please send text(to cancel send /cancel )")
            bot.register_next_step_handler(message, get_visibility);
        else:
            bot.send_message(message.from_user.id, f"Got it. Parameter is changed")

    elif message.text == '/cancel':
        bot.send_message(message.from_user.id, f"Canceling comand.")
    else:
        bot.send_message(message.from_user.id, f"Thats not a text type. Please send text(to cancel send /cancel )")
        bot.register_next_step_handler(message, get_visibility);

#@bot.message_handler(content_types=['photo'])
def get_image(message):

    if message.content_type == 'photo':
        raw = message.photo[-1].file_id #max image size id
        path = raw+".jpg"
        file_info = bot.get_file(raw)
        downloaded_file = bot.download_file(file_info.file_path)
        with open(path,'wb') as new_file:
            new_file.write(downloaded_file)
            
        bot.send_message(message.from_user.id, f"Got image. Please wait..")#message.photo)
        #HERE IS PIC PROCCESSING
    elif message.text == '/cancel':
        bot.send_message(message.from_user.id, f"Canceling comand.")

        #bot.register_next_step_handler(message, );
    else:
        bot.send_message(message.from_user.id, f"I cant read this Image or it's not image. Please send image(to cancel send /cancel )")
        bot.register_next_step_handler(message, get_image);
        
    
    
# @bot.message_handler(content_types=['text', 'photo'])
# def get_message(message):
#     #print(type(message.content_type))
    
#    # bot.send_message(message.from_user.id, message.photo)
# #    print("lmao")
#     if message.content_type == 'photo':
#         bot.send_message(message.from_user.id, f"Launch counter: {str(launch_counter)}")#message.photo)
#         raw = message.photo[2].file_id
# #        print(raw, message.photo[file_id])
#         path = raw+".jpg"
#         file_info = bot.get_file(raw)
#         #print(file_info)
        
#         downloaded_file = bot.download_file(file_info.file_path)
#         #download()
#        # print(type(downloaded_file),downloaded_file)
#         with open(path,'wb') as new_file:
#             new_file.write(downloaded_file)
            
        
#    # else:
#    #     bot.send_message(message.from_user.id, "Я тебя не понимаю. Напиши /help.")
        
#         #    if message.text == "Привет":
#  #       bot.send_message(message.from_user.id, "Привет, чем я могу тебе помочь?")
#   #  elif message.text == "/help":
#    #     bot.send_message(message.from_user.id, "Напиши привет")
#     #else:
#      #   bot.send_message(message.from_user.id, "Я тебя не понимаю. Напиши /help.")

In [4]:
bot.polling(non_stop=False, interval=0)

2023-01-02 11:49:57,209 (apihelper.py:87 MainThread) DEBUG - TeleBot: "Request: method=get url=https://api.telegram.org/bot5639499556:{TOKEN}/getMe params=None files=None"
2023-01-02 11:49:57,558 (apihelper.py:160 MainThread) DEBUG - TeleBot: "The server returned: 'b'{"ok":true,"result":{"id":5639499556,"is_bot":true,"first_name":"Wound Analyzer","username":"WoundAnalyzerBot","can_join_groups":true,"can_read_all_group_messages":false,"supports_inline_queries":false}}''"
2023-01-02 11:49:57,559 (__init__.py:1040 MainThread) INFO - TeleBot: "Starting your bot with username: [@WoundAnalyzerBot]"
2023-01-02 11:49:57,560 (__init__.py:1057 MainThread) INFO - TeleBot: "Started polling."
2023-01-02 11:49:57,561 (util.py:97 PollingThread) DEBUG - TeleBot: "Received task"
2023-01-02 11:49:57,562 (apihelper.py:87 PollingThread) DEBUG - TeleBot: "Request: method=get url=https://api.telegram.org/bot5639499556:{TOKEN}/getUpdates params={'offset': 1, 'timeout': 20, 'long_polling_timeout': 20} files=N

2023-01-02 11:50:10,311 (__init__.py:664 PollingThread) DEBUG - TeleBot: "Received 1 new updates"
2023-01-02 11:50:10,312 (util.py:101 PollingThread) DEBUG - TeleBot: "Task complete"
2023-01-02 11:50:10,312 (util.py:97 WorkerThread1) DEBUG - TeleBot: "Received task"
2023-01-02 11:50:10,314 (util.py:97 PollingThread) DEBUG - TeleBot: "Received task"
2023-01-02 11:50:10,314 (apihelper.py:87 WorkerThread1) DEBUG - TeleBot: "Request: method=post url=https://api.telegram.org/bot5639499556:{TOKEN}/sendMessage params={'chat_id': '1099854951', 'text': 'Changing visibility. Enter num to replace visibility param:'} files=None"
2023-01-02 11:50:10,315 (apihelper.py:87 PollingThread) DEBUG - TeleBot: "Request: method=get url=https://api.telegram.org/bot5639499556:{TOKEN}/getUpdates params={'offset': 393609971, 'timeout': 20, 'long_polling_timeout': 20} files=None"
2023-01-02 11:50:10,653 (apihelper.py:160 WorkerThread1) DEBUG - TeleBot: "The server returned: 'b'{"ok":true,"result":{"message_id":21

visibility is chosen


2023-01-02 11:50:14,971 (apihelper.py:160 PollingThread) DEBUG - TeleBot: "The server returned: 'b'{"ok":true,"result":[{"update_id":393609971,\n"message":{"message_id":217,"from":{"id":1099854951,"is_bot":false,"first_name":"KOTOBOPOT","username":"KOTOBOPOT","language_code":"ru"},"chat":{"id":1099854951,"first_name":"KOTOBOPOT","username":"KOTOBOPOT","type":"private"},"date":1672649414,"text":"s"}}]}''"
2023-01-02 11:50:14,973 (__init__.py:664 PollingThread) DEBUG - TeleBot: "Received 1 new updates"
2023-01-02 11:50:14,973 (util.py:101 PollingThread) DEBUG - TeleBot: "Task complete"
2023-01-02 11:50:14,974 (util.py:97 WorkerThread1) DEBUG - TeleBot: "Received task"
2023-01-02 11:50:14,974 (util.py:97 PollingThread) DEBUG - TeleBot: "Received task"
2023-01-02 11:50:14,975 (apihelper.py:87 WorkerThread1) DEBUG - TeleBot: "Request: method=post url=https://api.telegram.org/bot5639499556:{TOKEN}/sendMessage params={'chat_id': '1099854951', 'text': 'Thats not a num type. Please send text(to

2023-01-02 11:52:02,393 (util.py:97 PollingThread) DEBUG - TeleBot: "Received task"
2023-01-02 11:52:02,393 (apihelper.py:87 PollingThread) DEBUG - TeleBot: "Request: method=get url=https://api.telegram.org/bot5639499556:{TOKEN}/getUpdates params={'offset': 393609974, 'timeout': 20, 'long_polling_timeout': 20} files=None"
2023-01-02 11:52:22,496 (apihelper.py:160 PollingThread) DEBUG - TeleBot: "The server returned: 'b'{"ok":true,"result":[]}''"
2023-01-02 11:52:22,498 (__init__.py:664 PollingThread) DEBUG - TeleBot: "Received 0 new updates"
2023-01-02 11:52:22,498 (util.py:101 PollingThread) DEBUG - TeleBot: "Task complete"
2023-01-02 11:52:22,500 (util.py:97 PollingThread) DEBUG - TeleBot: "Received task"
2023-01-02 11:52:22,501 (apihelper.py:87 PollingThread) DEBUG - TeleBot: "Request: method=get url=https://api.telegram.org/bot5639499556:{TOKEN}/getUpdates params={'offset': 393609974, 'timeout': 20, 'long_polling_timeout': 20} files=None"
2023-01-02 11:52:42,604 (apihelper.py:160 P

In [5]:
print({"123":"213"})

{'123': '213'}


In [6]:
print('a\na')

a
a


In [14]:
import configparser
config = configparser.ConfigParser()
config['DEFAULT'] = {
    "brightness":'0.5', 
    'visibility':'0.5',

    #Цвета в HEX формате
    'color1':'ecfc0a',#желтый
    'color2':'fad328',#светло-оранжевый
    'color3':'ff9000',#темно-оранжевый
    'color4':'fc0505',#красный
    
    'auto_apply_changes':'yes',
    'return_separate_predict':'no'
    
    }

#{'ServerAliveInterval': '45',
                    # 'Compression': 'yes',
                    # 'CompressionLevel': '9'}
# config['bitbucket.org'] = {}
# config['bitbucket.org']['User'] = 'hg'
# config['topsecret.server.com'] = {}
# topsecret = config['topsecret.server.com']
# topsecret['Port'] = '50022'     # mutates the parser
# topsecret['ForwardX11'] = 'no'  # same here
# config['DEFAULT']['ForwardX11'] = 'yes'
with open('config.ini', 'w') as configfile:
  config.write(configfile)

In [8]:
print(dict(config['DEFAULT']))

{'brightness': '0.5', 'visibility': '0.5', 'color1': 'ecfc0a', 'color2': 'fad328', 'color3': 'ff9000', 'color4': 'fc0505', 'auto_apply_changes': 'yes', 'return_separate_predict': 'no'}


In [9]:
settings_str = ""
for (key,item) in dict(config['DEFAULT']).items():
    settings_str+=f"{key}: {item}\n" #print(key,item)
print(settings_str)    

brightness: 0.5
visibility: 0.5
color1: ecfc0a
color2: fad328
color3: ff9000
color4: fc0505
auto_apply_changes: yes
return_separate_predict: no



In [10]:
a = dict(config['DEFAULT'])
list(a.keys())

['brightness',
 'visibility',
 'color1',
 'color2',
 'color3',
 'color4',
 'auto_apply_changes',
 'return_separate_predict']

In [11]:
class NNBOT:
    def __init__(self, token, is_logging = True):
        self.token = token
        self.bot = telebot.TeleBot(token);
        self.is_logging = self.__set_logging(is_logging)
        
        self.__set_up()
    def __set_logging(self,is_logging):
        self.is_logging = is_logging
        if is_logging:
            self.logger = telebot.logger
            telebot.logger.setLevel(logging.DEBUG) # Outputs debug messages to console.
    
    def __set_up(self):
    
        #@bot.message_handler(content_types=['text', 'photo'])
        @bot.message_handler(content_types=['text', 'photo'])
        def get_text_messages(message):
            if message.content_type == 'photo':
                bot.send_message(message.from_user.id, f"Launch counter: {str(launch_counter)}")#message.photo)
                raw = message.photo[2].file_id
                path = raw+".jpg"
                file_info = bot.get_file(raw)

                downloaded_file = bot.download_file(file_info.file_path)
                with open(path,'wb') as new_file:
                    new_file.write(downloaded_file)

    def start(self, non_stop = True, interval = 0, timeout = 20):
        ''' После вызова этой функции TeleBot начинает опрашивать серверы Telegram на предмет новых сообщений.
        none_stop: True / False (по умолчанию False) - не прекращать опрос при получении ошибки от серверов Telegram

        interval: True / False (по умолчанию False) - интервал между запросами на опрос. 
        изменение этого параметра снижает время отклика бота.

        timeout: целое число (по умолчанию 20) - Тайм-аут в секундах для длительного опроса
        '''
        self.bot.polling(non_stop=non_stop, interval=interval, timeout=timeout)
        
    